# Студент: Скачков Николай Андреевич
# Задание: Предложить и исследовать новые весовые схемы в задаче прогнозирования визитов пользователя

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
    
plt.style.use('ggplot')

%matplotlib inline

In [2]:
df = pd.read_csv('./training.csv')

In [3]:
df.head()

,customer_id,visit_date,visit_spend
0,2,2010-04-01,5.97
1,2,2010-04-06,12.71
2,2,2010-04-07,34.52
3,2,2010-04-12,7.89
4,2,2010-04-14,17.17


In [6]:
df_users = df.groupby(by=df.customer_id).agg({
    'visit_date': lambda x: ' '.join(x),
    'visit_spend': lambda x: ' '.join(map(str, x))
})

In [21]:
df_users_sample = df_users.loc[df_users.index[np.random.choice(np.arange(len(df_users)), size=10000)], :]

In [24]:
df_users_sample.head()

,visit_date,visit_spend
customer_id,,
27448,2010-04-05 2010-04-13 2010-04-26 2010-04-28 20...,150.54 2.0 29.92 84.97 14.8 20.19 174.91 20.69...
115768,2010-04-19 2010-04-20 2010-05-10 2010-05-12 20...,21.65 149.89 96.24 147.2 74.66 81.91 136.77 28...
60323,2010-04-03 2010-04-09 2010-04-13 2010-04-17 20...,35.44 102.75 30.21 4.18 8.33 19.17 51.5 7.78 2...
3753,2010-04-01 2010-04-03 2010-04-06 2010-04-07 20...,98.85 32.66 20.24 54.7 9.18 34.08 31.13 60.18 ...
78400,2010-04-04 2010-04-09 2010-04-18 2010-04-21 20...,4.45 56.18 53.59 4.52 67.8 16.0 10.0 16.94 44....


Рассмотрим следующие весовые схемы:

1) логарифмическая: $w_i = \frac{1}{ \log (i + 2) }$

2) среднее $w_i = 1$

3) среднее среди k последних: $w_i = 1, i < k$

4) линейная по k последним: $w_i = k - i, i < k$

Заметим, что все эти схемы после нормировки $w_i$ суммируются в единицу. Также, 1 и 4 схемы подразумевают, что более новые элементы более важны.  

В качестве основы для сравнения взята линейная весовая схема, описанная на лекции, с параметром $\delta = 1$.

Оценивать будем по способности предсказывать на основе данной схемы следующий приход пользователя в день с максимальной вероятностью его прихода.

Измерять качество будем с помощью метрики accuracy. 

In [182]:
def last_k_average_scema(values, k=10):
    return values[-k:].mean()


def last_k_linear_scema(values, k=10, alpha=1.0):
    val_slice = values[-k:]
    w = (np.arange(len(val_slice)) + 1) * alpha
    return np.sum(val_slice * w) / np.sum(w)


def log_schema(values):
    w = 1 / np.log(np.arange(len(values)) + 2)
    w = w[::-1]
    return np.sum(values * w) / w.sum()


def linear_schema(values):
    w = np.linspace(0, 1, num=len(values))
    return np.sum(values * w) / w.sum()


def average_schema(values):
    return np.mean(values)
    

def predict_using_schema(dates, spend, schema):
    dates = dates.split()
    spend = map(float, spend.split())
    dates, spend = zip(*sorted(zip(dates, spend)))
    dts = list(map(lambda x: datetime.strptime(x, "%Y-%m-%d"), dates))
    dayweeks = list(map(lambda x: x.weekday(), dts))
    weeks = list(map(lambda x: x.year * 100 + x.isocalendar()[1], dts))
    dayweeks = np.array(dayweeks)
    compressed_weeks = [0]
    for i in range(1, len(weeks)):
        if weeks[i] >= weeks[i-1] + 1:
            compressed_weeks.append(compressed_weeks[-1] + 1)
        else:
            compressed_weeks.append(compressed_weeks[-1])
    compressed_weeks = np.array(compressed_weeks)
    
    spend = np.array(spend)
    scores_w = []
    bins = []
    maximum = compressed_weeks.max()
    if maximum <= 1:
        return True
    for weekday in range(7):
        mask = dayweeks == weekday
        dayhist = spend[mask]
        visitshist = compressed_weeks[mask]
        binarized = np.zeros(maximum + 1)
        binarized[visitshist] = 1
        pr_vis = schema(binarized[:-1])
        scores_w.append(pr_vis)
        bins.append(binarized[-1])
      
    return (scores_w, bins)

In [183]:
from sklearn.metrics import roc_auc_score

In [186]:
schemas = [
    log_schema,
    last_k_average_scema,
    last_k_linear_scema,
    linear_schema,
    average_schema
]


results = dict()
for schema in schemas:
    all_scores_vis = []
    all_bins = []
    for dates, spend in zip(df_users_sample.visit_date.values, df_users_sample.visit_spend.values):
        scores, bins = predict_using_schema(dates, spend, schema=schema)
        all_scores_vis.extend(scores)
        all_bins.extend(bins)
    results[schema.__name__] = roc_auc_score(y_score=all_scores_vis, y_true=all_bins)

# Результаты

In [190]:
pd.DataFrame(results.items(), columns=['schema', 'ROC_AUC'])

,schema,ROC_AUC
0,last_k_average_scema,0.728660
1,last_k_linear_scema,0.724838
2,log_schema,0.753316
3,linear_schema,0.753372
4,average_schema,0.749942


Итоги: в приведённой таблице можно увидеть сравнение предложенных весовых схем со стандартной, описанной в лекции линейной схемой с параметром $\delta = 1$.

По результатам сравнения можно сделать вывод, что:

1) последние значения посещений клиента важнее более старых, так как average_schema уступает linear_schema

2) схемы, основанные на последних элементах уступают остальным по качеству. Это скорее всего связано с тем, что оценки, построенные на относительно небольшом количестве элементов более шумные

3) логарифмическая схема и описанная на лекции линейная, имеют приблизитнльо одинаковое качество, что говорит о том, что сила учитывания "хвоста" истории  не имеет принципиального решения в данной задаче. 